# Análise Descritiva dos Dados

## Título:

**Predição de preço de imóveis**

## Membros:

*   Adrisson Rogério Samersla
*   Nickolas Batista Mendonça Machado
*   Thayna Pires Baldão



# Setup

In [ ]:
# Instalando os pacotes necessários para a análise

!pip3 install geopandas mapclassify googledrivedownloader

In [ ]:
# Importando os pacotes necessários para a análise

import os

import pandas            as pd
import geopandas         as gpd
import numpy             as np
import scipy             as sc
import matplotlib.pyplot as plt
import seaborn           as sns

from google_drive_downloader import GoogleDriveDownloader as gdd

import mapclassify

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.reset_option('max_colwidth')

In [ ]:
# Baixando o dataset

dataset_dir = "../dataset"
has_dataset_dir = os.path.isdir(dataset_dir)
if (not has_dataset_dir):
  # Link of dataset folder: 
  # https://drive.google.com/file/d/1oymNGU7E7adnwqIkZ0W_zxcuzj-KFIqb/view?usp=sharing
  gdd.download_file_from_google_drive(file_id='1oymNGU7E7adnwqIkZ0W_zxcuzj-KFIqb',
                                      dest_path='../dataset.zip',
                                      showsize=True,
                                      unzip=True)

In [ ]:
# Criando pasta para as análises

images_dir = "./imagens/"
has_images_dir = os.path.isdir(images_dir)
if (not has_images_dir):
  os.mkdir(images_dir)

# Entendendo a Base de Dados

In [ ]:
# Lendo a base de dados

df = pd.read_csv(dataset_dir + '/dataset.csv')
print("Formato dos dados: ", df.shape)
print("#Exemplos: {}".format(df.shape[0]))
print("#Atributos: {}".format(df.shape[1]))

In [ ]:
# Inspecionando os atributos

df.dtypes

In [ ]:
# Inspecionando os exemplos

df.head()

In [ ]:
# Removendo os atributos `id` e `property_id` 
# pois são inúteis para a tarefa de aprendizado

X = df.drop(columns=['id','property_id'])
X.dtypes

# Tipo e Escala de Atributos

A seguir, cada atributo será detalhado conforme seu tipo e escala

## Qualitativos Nominais

### Atributo: operation

In [ ]:
pd.unique(X.operation)

In [ ]:
X.operation.astype('str').value_counts()

### Atributo: property_type

In [ ]:
pd.unique(X.property_type)

In [ ]:
X.property_type.astype('str').value_counts()

### Atributo: place_name

In [ ]:
pd.unique(X.place_name)

In [ ]:
X.place_name.astype('str').value_counts()

### Atributo: place_with_parent_names

In [ ]:
pd.unique(X.place_with_parent_names)

In [ ]:
X.place_with_parent_names.astype('str').value_counts()

### Atributo: country_name

In [ ]:
pd.unique(X.country_name)

In [ ]:
X.country_name.astype('str').value_counts()

### Atributo: state_name

In [ ]:
pd.unique(X.state_name)

In [ ]:
X.state_name.astype('str').value_counts()

### Atributo: currency

In [ ]:
pd.unique(X.currency)

In [ ]:
X.currency.astype('str').value_counts()

### Atributo: description

In [ ]:
pd.unique(X.description)

### Atributo: title

In [ ]:
pd.unique(X.title)

### Atributo: thumbnail

In [ ]:
pd.unique(X.image_thumbnail)

## Quantitativos Intervalares

In [ ]:
pd.unique(X.created_on)

In [ ]:
pd.unique(X.collected_on)

## Quantitativos Racionais

In [ ]:
pd.unique(X.lat_lon)

# Análises Univariadas

## Valores Ausentes

In [ ]:
# Porcentagem de valores nulos no dataset

df.isnull().sum()*100/1_000_000

In [ ]:
# Analisando o atributo geonames_id

df.geonames_id.astype('str').value_counts()

## Distribuição dos Valores Numéricos

In [ ]:
# Algumas métricas para os atributos numéricos

X.drop(columns=['geonames_id']).describe()

## Distribuição Geográfica

In [ ]:
# Histograma dos estados

print(X.state_name.astype('str').value_counts())
print()

fig, ax = plt.subplots(figsize=(20,5))
ax = sns.histplot(data=X, x='state_name', stat='probability', shrink=.8, ax=ax)
plt.xticks(rotation=90)
plt.show()
fig.savefig(images_dir + 'hist_states.png', format='png')

In [ ]:
# Valores extremos de latitude e longitude

vals = X.lat.dropna()
print("Latitude: ")
print("Min: ", vals.min())
print("Max: ", vals.max())

print()

vals = X.lon.dropna()
print("Longitude: ")
print("Min: ", vals.min())
print("Max: ", vals.max())

In [ ]:
# Plotando a posição dos imóveis no mapa do Brasil

map_file = dataset_dir + '/GIS_UFEBRASIL/UFEBRASIL.shp'
map_df = gpd.read_file(map_file)

fig, ax = plt.subplots(figsize=(20,20))
ax = map_df.plot(color='white', edgecolor='black', ax=ax)


data = X[['lat', 'lon']]
data['price'] = np.log10(X.price)
data = data.dropna()

lon_max = -30.0
lon_min = -80.0
lat_min = -40.0
lat_max = 10.0
data = data.loc[((data.lat > lat_min) & (data.lat < lat_max)) & ((data.lon > lon_min) & (data.lon < lon_max))]

gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data.lon, data.lat))
# classifier = mapclassify.Quantiles(gdf.price, k=5)

# gdf.plot(ax=ax, column='price', cmap='Greens', markersize=1, scheme=classifier)
gdf.plot(ax=ax, column='price', markersize=10, alpha=0.1, cmap='OrRd', scheme='quantiles')

plt.title('Localização dos imóveis', fontsize=18)
plt.xlabel('Longitude', fontsize=18)
plt.ylabel('Latitude', fontsize=18)
fig.savefig(images_dir + 'mapa.png', format='png')

## Distribuição das Datas

In [ ]:
# A data precisa ser, primeiramente, parseada para um formato numérico

def parse(line):
    parts = line.split('-')
    return '{}-{}'.format(parts[0], parts[1])

def format(date):
    return "{}/{}".format(date.month, date.year)

dates = X.created_on.apply(parse)
dates = pd.to_datetime(dates)

start = dates.min()
end = dates.max()
seq = pd.date_range(start=start, end=end, freq="M")

print('Min: ', format(start))
print('Max: ', format(end))
print('#Months: ', len(seq))

dates = pd.DataFrame({'datetime': dates.apply(format)})

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax = sns.histplot(data=dates, x='datetime', stat='probability', shrink=.8, ax=ax)

labels = map(format, seq.to_list())
ax.set_xticklabels(list(labels), rotation=90)

plt.title('Datas de criação dos anúncios')
plt.xlabel('Mês')
plt.ylabel('Fração dos objetos')
plt.show()
fig.savefig(images_dir + 'created_on.png', format='png')

## Distribuição dos tipos de propriedade

In [ ]:
data = X.property_type.value_counts()
print(data)

print()

pie, ax = plt.subplots(figsize=[5,5])
labels = data.keys()
plt.pie(x=data, autopct="%.1f%%", explode=[0.05]*4, labels=labels, pctdistance=0.5)
plt.title("Tipos de propriedades", fontsize=14);
pie.savefig(images_dir + "property_type.png")

## Distribuição das Áreas

In [ ]:
data = X[['surface_total_in_m2', 'surface_covered_in_m2']].dropna()
data.rename(columns = {'surface_total_in_m2': 'total', 'surface_covered_in_m2': 'contruída'}, inplace=True)
print("Valores de Skew: ")
print("total    : ", sc.stats.skew(data['total']))
print("contruída: ", sc.stats.skew(data['contruída']))

print()

print("Valores máximos: ")
print(data.max())

print()

print("Valores ordenados por área total:")
print(data.sort_values(by=['total'], ascending=True).head(n=5))
print(data.sort_values(by=['total'], ascending=True).tail(n=5))

print()

print("Valores ordenados por área construíída:")
print(data.sort_values(by=['contruída'], ascending=True).head(n=5))
print(data.sort_values(by=['contruída'], ascending=True).tail(n=5))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(data=data.apply(np.log10), orient='h', palette="Set2", ax=ax)

plt.title('Boxplots do tamanho total do imóvel e da área construída.')
plt.xlabel('Log Tamanho')
plt.show()
fig.savefig(images_dir + 'boxplot_surfaces.png', format='png')

## Distribuição dos Quartos 

In [ ]:
print(X.rooms.astype('str').value_counts())

print()

data = X['rooms'].dropna()
print('Max(rooms)', data.max())
print('Min(rooms)', data.min())

In [ ]:
data = X[['rooms']].dropna()
fig, ax = plt.subplots(figsize=(10,5))
ax = sns.histplot(data=data, x='rooms', stat='probability', bins=40, shrink=0.8, ax=ax)

plt.title('Histograma de Quartos')
plt.xlabel('Número de Quartos')
plt.ylabel('Fração dos objetos')
plt.show()
fig.savefig(images_dir + 'hist_rooms.png', format='png')

## Distribuição dos Andares

In [ ]:
print(X.floor.astype('str').value_counts())

print()

print("Porcentagem de valores presentes: ", 
      X.floor.dropna().count()*100/1_000_000, "%")

print("Porcentagem de valores acima de 5 andares: ", 
      (X['floor'] > 5.0).sum()*100/1_000_000, "%")

print()

print('Max(floor)', X.floor.max())
print('Min(floor)', X.floor.min())

## Distribuição das Despesas e do Preço

In [ ]:
print("Despesas: ")
print(X.expenses.astype('str').value_counts())

print()

print("Preço: ")
print(X.price.astype('str').value_counts())

print()

print('Max(expenses)', X.expenses.max())
print('Max(price)', X.price.max())
print('Min(expenses)', X.expenses.min())
print('Min(price)', X.price.min())

In [ ]:
data = X[['expenses', 'price']].dropna()
data.rename(columns={'expenses': 'despesas', 'price': 'preço'}, inplace=True)

fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(data=data.apply(np.log10), orient='h', palette="Set2", ax=ax)

plt.title('Boxplots das despesas e do custo do imóvel')
plt.xlabel('Log Valor')
plt.show()
fig.savefig(images_dir + 'boxplot_price.png', format='png')

# Análises Multivariadas

## Correlação

In [ ]:
# Selecionando os atributos numééricos desejados

Xnumeric = X.select_dtypes(include=['int64', 'float64'])
Xnumeric = Xnumeric.drop(columns = ['geonames_id','floor'])
Xnumeric.shape

In [ ]:
# Matriz de correlação (Pearson)

Xnumeric.corr()

In [ ]:
# Matriz de correlação (Spearman)

plot = sns.heatmap(Xnumeric.corr(method='spearman'), annot=True)
plot.figure.tight_layout()
plot.figure.savefig(images_dir + 'heatmap.eps')